In [49]:
import re
import sys

In [50]:
delimiters = [',', ';', '/', '\\', ' ']
re_operaters = r"[*|-|/|+|%|=|<|>|^<=$|^>=$]"
re_preemtive_types = r"^boolean$|^byte$|^char$|^short$|^int$|^long$|^float$|^double$"
re_non_preemtive_types = r"^string$|^array$|^class$"
re_keywords = r"^var$|^and$|^or$|^not$|^if$|^elif$|^else$|^for$|^to$|^step$|^while$|^fun$|^then$|^end$|^return$|^continue$|^break$"

re_float_number = r"[+-]?[0-9]+[.,][0-9]+"
re_integer_number = r"[+-]?[0-9]"
re_string = r"[A-Za-z0-9_./\-]*"
re_char = r"\W[0-9]\W|\W[a-zA-Z]\W"

In [51]:
def ifOperator(word):
  if re.match(re_operaters, word):
    return True
  return False

def ifPreemtiveType(word):
  if re.match(re_preemtive_types, word):
    return True
  return False

def ifNonPreemtiveType(word):
  if re.match(re_non_preemtive_types, word):
    return True
  return False


def ifKeyword(word):
  if re.match(re_keywords, word):
    return True
  return False


def ifFloat(word):
  if re.match(re_float_number, word):
    return True
  return False


def ifInteger(word):
  if re.match(re_integer_number, word):
    # if re.search(r"[a-zA-Z]|\W[0-9]\W",word):
    # return False
    return True
  return False


def ifString(word):
  if re.match(re_string, word):
    return True
  return False


def ifChar(word):
  if re.match(re_char, word):
    return True
  return False

def dfa(word, index_of_word, line_number):
  global flag_temp
  if index_of_word == 0:
    if ifKeyword(word):
      tokens.append(['KEYWORD', word])
      tokens_list.append((word, line_number, 'KEYWORD'))
      return 
    if ifPreemtiveType(word): 
      tokens.append(['DATATYPE', word])
      tokens_list.append((word, line_number, 'DATATYPE'))
      return 
    if ifNonPreemtiveType(word):
      tokens.append(['DATATYPE', word])
      tokens_list.append((word, line_number, 'DATATYPE'))
      return 
    print("ERORR at line #{}: TYPO[DATATYPE, KEYWORD]. [{}]".format(i+1, word))
    #tokens.append(['DATATYPE', word])
    flag_temp = True
    return 
    #sys.exit("ERORR at line #{}: TYPO. [{}]".format(i+1, word))

  # DELIMITER.
  deli_flag = False
  for delimiter in delimiters:
    if word[len(word) - 1] == delimiter:
      #print("ITS A DELIMITER", word)
      delimiter_type = delimiter
      deli_flag = True
  
  # identify identifiers.
  if flag_temp == True:
    if re.match("[a-z]|[A-Z]", word):
      tokens.append(['IDENTIFIER', word])
      tokens_list.append((word, line_number, 'IDENTIFIER'))
      flag_temp = False
      return 
    else:
      print("ERORR at line #{}: INVALID IDENTIFIER NAME[IDENTIFIER]. [{}]".format(i+1, word))
      return 
      #sys.exit("ERORR at line #{}: INVALID IDENTIFIER NAME. [{}]".format(i+1, word))

  if tokens[len(tokens) - 1][0] == 'DATATYPE':
    if re.match("[a-z]|[A-Z]", word):
      tokens.append(['IDENTIFIER', word])
      tokens_list.append((word, line_number, 'IDENTIFIER'))
      return 
    else:
      print("ERORR at line #{}: INVALID IDENTIFIER NAME[IDENTIFIER]. [{}]".format(i+1, word))
      return 
      #sys.exit("ERORR at line #{}: INVALID IDENTIFIER NAME. [{}]".format(i+1, word))

  # identify operators.
  if ifOperator(word) == True:
    tokens.append(['OPERATOR', word])
    tokens_list.append((word, line_number, 'OPERATOR'))
    return 

  # identify FLOAT.
  if ifFloat(word):
    if deli_flag == True:
      tokens.append(["FLOAT", word[:-1]])
      tokens_list.append((word[:-1], line_number, 'FLOAT'))
      tokens.append(['END_STATEMENT', delimiter_type])
      tokens_list.append((word[len(word) - 1], line_number, 'END_STATEMENT'))
      return 
    else:
      tokens.append(["FLOAT", word])
      tokens_list.append((word, line_number, 'FLOAT'))
      return 
         
   # identify integer.
  if ifInteger(word):
    if deli_flag == True:
      tokens.append(["INTEGER", word[:-1]])
      tokens_list.append((word[:-1], line_number, 'INTEGER'))
      tokens.append(['END_STATEMENT', delimiter_type])
      tokens_list.append((word[len(word) - 1], line_number, 'END_STATEMENT'))
      return 
    else:
      tokens.append(["INTEGER", word])
      tokens_list.append((word, line_number, 'INTEGER'))
      return 
      
  # identify Character.
  if ifChar(word):
    if deli_flag == True:
      tokens.append(["CHARACTER", word[:-1]])
      tokens_list.append((word[:-1], line_number, 'CHARACTER'))
      tokens.append(['END_STATEMENT', delimiter_type])
      tokens_list.append((word[len(word) - 1], line_number, 'END_STATEMENT'))
      return 
    else:
      tokens.append(["CHARACTER", word])
      tokens_list.append((word, line_number, 'CHARACTER'))
      return 

   # identify STRING
  if ifString(word):
    if deli_flag == True:
      tokens.append(["STRING", word[:-1]])
      tokens_list.append((word[:-1], line_number, 'STRING'))
      tokens.append(['END_STATEMENT', delimiter_type])
      tokens_list.append((word[len(word) - 1], line_number, 'END_STATEMENT'))
      return 
    else:
      tokens.append(["STRING", word])
      tokens_list.append((word, line_number, 'STRING'))
      return 
  print("ERORR at line #{}: ILLEGAL CHARACTER. [{}]".format(i+1, word))
  return True
  #sys.exit("ERORR at line #{}: INVALID IDENTIFIER NAME. [{}]".format(i+1, word))

In [52]:
f = open('test.txt', 'r+')
contents = f.readlines()
f.close()

global flag_temp
flag_temp = False


tokens = []
tokens_list = []
counter = 0
for i in range(len(contents)):
  contents_at_line = contents[i].split()
  for word in contents_at_line:
    dfa(word, contents_at_line.index(word), i+1)
      
      
  print('--> Line #{}:'.format(i+1), end = ' ')
  print(tokens[counter:])
  counter = len(tokens)
  
print("PROGRAM FINISHED...")

--> Line #1: [['DATATYPE', 'int'], ['IDENTIFIER', 'i'], ['OPERATOR', '='], ['INTEGER', '50'], ['END_STATEMENT', ';']]
ERORR at line #2: TYPO[DATATYPE, KEYWORD]. [intt]
--> Line #2: [['IDENTIFIER', 'i'], ['OPERATOR', '='], ['INTEGER', '50'], ['END_STATEMENT', ';']]
--> Line #3: [['DATATYPE', 'string'], ['IDENTIFIER', 's'], ['OPERATOR', '='], ['STRING', '"hello"'], ['END_STATEMENT', ';']]
--> Line #4: [['KEYWORD', 'for']]
ERORR at line #5: TYPO[DATATYPE, KEYWORD]. [tor]
--> Line #5: []
ERORR at line #6: TYPO[DATATYPE, KEYWORD]. [zx]
--> Line #6: []
PROGRAM FINISHED...


In [53]:
from tkinter import *
  
class Table:

	def __init__(self,root):
		# code for creating table
		for i in range(total_rows):
			for j in range(total_columns):
				self.e = Entry(root, width=20, fg='blue', font=('Arial',16,'bold'))
				self.e.grid(row=i, column=j)
				self.e.insert(END, tokens_list[i][j])

# take the data
#lst = [(1,'Raj','Mumbai',19), (2,'Aaryan','Pune',18), (3,'Vaishnavi','Mumbai',20), (4,'Rachna','Mumbai',21), (5,'Shubham','Delhi',21)]

# find total number of rows and
# columns in list
total_rows = len(tokens_list)
total_columns = len(tokens_list[0])

# create root window
root = Tk()
t = Table(root)
root.mainloop()


TypeError: Table() takes no arguments

old code vvv

In [ ]:
''' f = open('test.txt', 'r+')
contents = f.readlines()
f.close()

tokens = []
counter = 0
for i in range(len(contents)):
  contents_at_line = contents[i].split()
  for word in contents_at_line:
    if word == contents_at_line[0]:
      if ifKeyword(word):
        tokens.append(['KEYWORD', word])
        continue
      if ifPreemtiveType(word): 
        tokens.append(['DATATYPE', word])
        continue
      if ifNonPreemtiveType(word):
        tokens.append(['DATATYPE', word])
        continue

      print("ERORR at line %d: TYPO." %i, word)

    # DELIMITER.
    deli_flag = False
    for delimiter in delimiters:
      if word[len(word) - 1] == delimiter:
        #print("ITS A DELIMITER", word)
        delimiter_type = delimiter
        deli_flag = True
      
    # identify identifiers.
    if re.match("[a-z]|[A-Z]", word):
      tokens.append(['IDENTIFIER', word])
      continue
    
    # identify operators.
    if ifOperator(word) == True:
      tokens.append(['OPERATOR', word])
      continue
    
    # identify FLOAT.
    if ifFloat(word):
      if deli_flag == True:
        tokens.append(["FLOAT", word[:-1]])
        tokens.append(['END_STATEMENT', delimiter_type])
        continue
      else:
        tokens.append(["FLOAT", word])
        continue
        
    # identify integer.
    if ifInteger(word):
      if deli_flag == True:
        tokens.append(["INTEGER", word[:-1]])
        tokens.append(['END_STATEMENT', delimiter_type])
        continue
      else:
        tokens.append(["INTEGER", word])
        continue

     # identify Character.
    if ifChar(word):
      if deli_flag == True:
        tokens.append(["CHARACTER", word[:-1]])
        tokens.append(['END_STATEMENT', delimiter_type])
        continue
      else:
        tokens.append(["CHARACTER", word])
        continue 

    # identify STRING
    if ifString(word):
      if deli_flag == True:
        tokens.append(["STRING", word[:-1]])
        tokens.append(['END_STATEMENT', delimiter_type])
        continue
      else:
        tokens.append(["STRING", word])
        continue

  print('Line %d:' %i, end = ' ')
  print(tokens[counter:])
  counter = len(tokens) '''

' f = open(\'test.txt\', \'r+\')\ncontents = f.readlines()\nf.close()\n\ntokens = []\ncounter = 0\nfor i in range(len(contents)):\n  contents_at_line = contents[i].split()\n  for word in contents_at_line:\n    if word == contents_at_line[0]:\n      if ifKeyword(word):\n        tokens.append([\'KEYWORD\', word])\n        continue\n      if ifPreemtiveType(word): \n        tokens.append([\'DATATYPE\', word])\n        continue\n      if ifNonPreemtiveType(word):\n        tokens.append([\'DATATYPE\', word])\n        continue\n\n      print("ERORR at line %d: TYPO." %i, word)\n\n    # DELIMITER.\n    deli_flag = False\n    for delimiter in delimiters:\n      if word[len(word) - 1] == delimiter:\n        #print("ITS A DELIMITER", word)\n        delimiter_type = delimiter\n        deli_flag = True\n      \n    # identify identifiers.\n    if re.match("[a-z]|[A-Z]", word):\n      tokens.append([\'IDENTIFIER\', word])\n      continue\n    \n    # identify operators.\n    if ifOperator(word) == 

In [ ]:
s='boolean' , 'byte' , 'char' , 'short' , 'int', 'long' , 'float', 'double'